Installing the Julep Client

In [1]:
!pip install --upgrade julep --quiet

In [2]:
# Global UUID is generated for agent and task
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4() 

## creating julep client with the api key


In [3]:
from julep import Client
import os

JULEP_API_KEY = os.environ['JULEP_API_KEY']

# Create a client
client = Client(api_key=JULEP_API_KEY, environment="production")

### Creating an "agent"

Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the Agent section in [Julep Concepts](https://docs.julep.ai/docs/concepts/agents).

In [4]:
# Defining the agent
name = "Chad"
about = "Sarcastic news headline reporter."

# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="claude-3.5-sonnet",
)

### Defining a Task

Tasks in Julep are Github-Actions-style workflows that define long-running, multi-step actions.

You can use them to conduct complex actions by defining them step-by-step.

To learn more about tasks, please refer to the `Tasks` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/tasks).

In [5]:
import yaml

BRAVE_API_KEY = "YOUR_BRAVE_API_KEY"

# Define the task
task_def = yaml.safe_load(f"""
# yaml-language-server: $schema=https://raw.githubusercontent.com/julep-ai/julep/refs/heads/dev/src/schemas/create_task_request.json
name: Julep Sarcasm Headline Generator
description: This task generates a sarcastic news headline on a user-provided topic.

########################################################
####################### INPUT SCHEMA ###################
########################################################

input_schema:
  type: object
  properties:
    topic:
      type: string
      description: The topic of the news headline

########################################################
####################### TOOLS ###########################
########################################################

# Define the tools that the task will use in this workflow
tools:
- name: brave_search
  type: integration
  integration:
    provider: brave
    setup:
      brave_api_key: "{BRAVE_API_KEY}"

########################################################
####################### MAIN WORKFLOW ##################
########################################################

main:
# Step 0: Call the brave_search tool with the topic input
- tool: brave_search
  arguments:
    query: $ _.topic + ' funny news'

# Step 1: Evaluate step to create a summary of the results
- evaluate:
    search_results: |-
      $ [
        {{
          'snippet': r['snippet'],
          'title': r['title']
        }}
        for r in _['result']
      ]

# Step 2: Prompt step to create a summary of the results
- prompt:
  - role: system
    content: >-
      $ f'''You are {{agent.about}}.
      The user will send you a topic and search results for that topic.
      Your goal is to write a sarcastic news headlines based on that topic and search results.'''
  - role: user
    content: >-
      $ f'''My topic is: {{steps[0].input.topic}}.
      Here are the search results: {{_}}'''
  unwrap: true

""")

In [6]:
# creating the task object
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

### Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

To learn more about executions, please refer to the `Executions` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/execution).

In [7]:
# creating an execution object
execution = client.executions.create(
    task_id=TASK_UUID,
    input={
        "topic": "Elon Musk"
    }
)

## Checking execution details and output

There are multiple ways to get the execution details and the output:

1. **Get Execution Details**: This method retrieves the details of the execution, including the output of the last transition that took place.

2. **List Transitions**: This method lists all the task steps that have been executed up to this point in time, so the output of a successful execution will be the output of the last transition (first in the transition list as it is in reverse chronological order), which should have a type of `finish`.


<span style="color:olive;">Note: You need to wait for a few seconds for the execution to complete before you can get the final output, so feel free to run the following cells multiple times until you get the final output.</span>


In [8]:
import time

execution = client.executions.get(execution.id)

while execution.status != "succeeded":
    time.sleep(5)
    execution = client.executions.get(execution.id)
    print("Execution status: ", execution.status)
    print("-"*50)

execution = client.executions.get(execution.id)

print(execution.output)

"Genius Billionaire Desperately Attempts Comedy, Proves Money Can't Buy Sense of Humor"

"Breaking: World's Richest Man Still Can't Figure Out How to Make People Genuinely Like Him"

"Shocking Report: Man Who Named Child 'X Æ A-12' May Not Be Most Stable CEO"

"Former Trump Officials Suggest Drug Tests for Musk; Musk Agrees, Proving He Might Actually Need One"

"Billionaire Space Enthusiast Too Busy Being 'Funny' on Twitter to Notice Everyone's Cringing"

"Studies Show Being Worth $180 Billion Still Won't Make Your Memes Any Better"

"Man Who Could End World Hunger Chooses to Post Dad Jokes on Social Media Instead"

"Breaking: Local Attention-Seeking Billionaire Still Seeking Attention, More at 11"


In [9]:
# Lists all the task steps that have been executed up to this point in time
transitions = client.executions.transitions.list(execution_id=execution.id).items

# Transitions are retrieved in reverse chronological order
for transition in reversed(transitions):
    print("Transition type: ", transition.type)
    print("Transition output: ", transition.output)
    print("-"*50)

Transition type:  init
Transition output:  {'topic': 'Elon Musk'}
--------------------------------------------------
Transition type:  step
Transition output:  {'result': [{'link': 'https://9gag.com/tag/elon-musk', 'title': 'Best Elon Musk Memes and Images - 9GAG', 'snippet': 'Enjoy the best of new <strong>funny</strong> cursed meme images about <strong>Elon</strong> <strong>Musk</strong>, Twitter, $8 verified badge, Tesla, the Boring Company and more on 9GAG. Never run out of hilarious cursed memes to share.'}, {'link': 'https://www.youtube.com/watch?v=ukxf9AqLsPY', 'title': 'Elon FINALLY Gets the HUMILIATION He Deserves in Front of MILLIONS - YouTube', 'snippet': 'Comedian Bill Burr roasts <strong>Elon</strong> <strong>Musk</strong>, MAGA melts down over SNL, and protesters hit Trump where it hurts—his billionaire bestie&#x27;s wallet. Gabe Sanchez reports ...'}, {'link': 'https://gizmodo.com/elon-musk-is-annoying-unfunny-and-should-probably-take-a-drug-test-trump-officials-reportedl

## Running the same task with a different topic

We will use the same code to run the same task, but with a different topic

In [10]:
execution = client.executions.create(
    task_id=TASK_UUID,
    input={
        "topic": "Tottenham Hotspur"
    }
)

In [11]:
import time

execution = client.executions.get(execution.id)

while execution.status != "succeeded":
    time.sleep(5)
    execution = client.executions.get(execution.id)
    print("Execution status: ", execution.status)
    print("-"*50)

execution = client.executions.get(execution.id)

print(execution.output)

Execution status:  succeeded
--------------------------------------------------
"In Shocking Display of Stockholm Syndrome, Former Arsenal Executive Defects to Tottenham"

"Desperate Spurs Hire Ex-Arsenal CEO, Proving There's No Rock Bottom They Won't Explore"

"Local Man Voluntarily Leaves Success to Join Eternal Trophy Drought: A London Tale"

"Tottenham Finally Wins Something: The Heart of Arsenal's Former CEO"

"North London Rivalry Takes Bizarre Turn as Spurs Resort to Executive Poaching Instead of Trophy Hunting"

Pick any of these sarcastic headlines that capture the irony of Tottenham hiring their rival Arsenal's former CEO, while playfully jabbing at Spurs' notorious trophy drought and the historic rivalry between the two clubs! 😏


<span style="color:olive;">Note: you can get the output of the search step by accessing the corresponding transition's output from the transitions list.</span>

Example:

In [12]:
transitions = client.executions.transitions.list(execution_id=execution.id).items

transitions[1].output

{'search_results': [{'title': 'Tottenham Hotspur FC - latest news, pictures, video comment - Football.london',
   'snippet': 'The latest <strong>Tottenham</strong> <strong>news</strong>, transfers, fixtures and more. Including Live blogs, pictures, video, podcasts, polls and indepth analysis from our dedicated Spurs writers.'},
  {'title': 'Spurs Media Watch, News and Stories',
   'snippet': 'These stories have been specially selected from today&#x27;s media. They do not necessarily represent the views or position of <strong>Tottenham</strong> <strong>Hotspur</strong> Football Club. For total Spurs <strong>news</strong> coverage, visit NewsNow.co.uk, the UK&#x27;s #1 football <strong>news</strong> aggregator.'},
  {'title': 'Tottenham Hotspur FC - Transfer news, results, fixtures, video and audio',
   'snippet': 'Having left Arsenal at the end of the 2023-24 campaign, Venkatesham has bided his time before returning to football. <strong>Tottenham confirmed Venkatesham&#x27;s appointment

## Related Concepts

- [Agents](https://docs.julep.ai/concepts/agents)
- [Tasks](https://docs.julep.ai/concepts/tasks)
- [Tools](https://docs.julep.ai/concepts/tools)
